# MMS Magnetopause Analysis - Basic Usage

This notebook demonstrates the basic usage of the MMS Magnetopause Analysis Toolkit.

## Overview

We'll analyze a magnetopause crossing event from January 27, 2019, showing:
1. Data loading from CDAWeb
2. Coordinate transformations
3. Boundary detection
4. Basic visualization

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Import MMS-MP toolkit
import mms_mp

# Configure matplotlib for inline plots
%matplotlib inline
plt.style.use('default')

print(f"MMS-MP version: {mms_mp.__version__}")

## 1. Configuration

Set up the analysis parameters:

In [ ]:
# Analysis configuration
trange = ['2019-01-27/12:20:00', '2019-01-27/12:40:00']
probes = ['1', '2']  # Start with just 2 spacecraft for simplicity
cadence = '150ms'

print(f"Time range: {trange[0]} to {trange[1]}")
print(f"Spacecraft: MMS{', MMS'.join(probes)}")
print(f"Cadence: {cadence}")

## 2. Data Loading

Load MMS data from CDAWeb. This may take a few minutes on first run as data is downloaded and cached.

In [ ]:
# Load MMS data
print("Loading MMS data...")
evt = mms_mp.load_event(
    trange, 
    probes=probes,
    include_edp=False,  # Skip electric field for simplicity
    include_ephem=True
)
print("✓ Data loaded successfully")

# Show what data we have
probe = probes[0]
print(f"\nData variables for MMS{probe}:")
for key in evt[probe].keys():
    time, data = evt[probe][key]
    print(f"  {key}: {data.shape} ({len(time)} time points)")

## 3. Data Processing

Process the data for boundary detection:

In [ ]:
# Process MMS1 data
probe = '1'
d = evt[probe]

# Resample to uniform grid
t_grid, vars_grid, good = mms_mp.merge_vars({
    'Ni': (d['N_tot'][0], d['N_tot'][1]),
    'Ne': (d['N_e'][0], d['N_e'][1]),
    'He': (d['N_he'][0], d['N_he'][1]),
    'B': (d['B_gsm'][0], d['B_gsm'][1]),
}, cadence=cadence, method='linear')

print(f"Resampled to {len(t_grid)} points at {cadence} cadence")
print(f"Time range: {datetime.utcfromtimestamp(t_grid[0]):%H:%M:%S} to {datetime.utcfromtimestamp(t_grid[-1]):%H:%M:%S}")

## 4. Coordinate Transformation

Transform to boundary-normal coordinates using the hybrid LMN method:

In [ ]:
# LMN coordinate transformation
from scipy.interpolate import interp1d

# Use middle portion of magnetic field for MVA
mid_idx = len(d['B_gsm'][0]) // 2
B_slice = d['B_gsm'][1][mid_idx-64:mid_idx+64, :3]

# Get spacecraft position at middle time
t_mid = d['B_gsm'][0][mid_idx]
interp_pos = interp1d(d['POS_gsm'][0], d['POS_gsm'][1],
                     axis=0, bounds_error=False,
                     fill_value='extrapolate')
pos_mid = interp_pos(t_mid)

# Hybrid LMN transformation
lmn = mms_mp.hybrid_lmn(B_slice, pos_gsm_km=pos_mid)
B_lmn = lmn.to_lmn(vars_grid['B'][:, :3])
BN = B_lmn[:, 2]  # Normal component

print(f"LMN transformation complete")
print(f"L direction: [{lmn.L[0]:.3f}, {lmn.L[1]:.3f}, {lmn.L[2]:.3f}]")
print(f"M direction: [{lmn.M[0]:.3f}, {lmn.M[1]:.3f}, {lmn.M[2]:.3f}]")
print(f"N direction: [{lmn.N[0]:.3f}, {lmn.N[1]:.3f}, {lmn.N[2]:.3f}]")

## 5. Boundary Detection

Detect magnetopause crossings using He+ density and B_N rotation:

In [ ]:
# Boundary detection
cfg = mms_mp.DetectorCfg(he_in=0.25, he_out=0.15, BN_tol=1.0)
mask_all = good['He'] & good['B']

layers = mms_mp.detect_crossings_multi(
    t_grid, vars_grid['He'], BN,
    cfg=cfg, good_mask=mask_all
)

# Extract crossing times
from mms_mp.boundary import extract_enter_exit
xings = extract_enter_exit(layers, t_grid)

if xings:
    print(f"✓ {len(xings)} boundary crossing(s) detected:")
    for i, (t_cross, cross_type) in enumerate(xings):
        print(f"  {i+1}. {cross_type} at {datetime.utcfromtimestamp(t_cross):%H:%M:%S}")
else:
    print("No boundary crossings detected")

## 6. Visualization

Create a summary plot showing the key parameters:

In [ ]:
# Create time array for plotting (convert to hours from start)
t_hours = (t_grid - t_grid[0]) / 3600

# Create summary plot
fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

# Panel 1: Magnetic field components
axes[0].plot(t_hours, B_lmn[:, 0], 'b-', label='B_L', linewidth=1)
axes[0].plot(t_hours, B_lmn[:, 1], 'g-', label='B_M', linewidth=1)
axes[0].plot(t_hours, B_lmn[:, 2], 'r-', label='B_N', linewidth=1)
axes[0].set_ylabel('B [nT]')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_title(f'MMS{probe} Magnetopause Analysis')

# Panel 2: Ion densities
axes[1].semilogy(t_hours, vars_grid['Ni'], 'b-', label='N_i', linewidth=1)
axes[1].semilogy(t_hours, vars_grid['Ne'], 'r-', label='N_e', linewidth=1)
axes[1].set_ylabel('Density [cm⁻³]')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Panel 3: He+ density with detection thresholds
axes[2].plot(t_hours, vars_grid['He'], 'orange', linewidth=1.5, label='He⁺')
axes[2].axhline(cfg.he_in, color='red', linestyle='--', alpha=0.7, label=f'In threshold ({cfg.he_in})')
axes[2].axhline(cfg.he_out, color='blue', linestyle='--', alpha=0.7, label=f'Out threshold ({cfg.he_out})')
axes[2].set_ylabel('He⁺ density [cm⁻³]')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Panel 4: B_N with crossing markers
axes[3].plot(t_hours, BN, 'purple', linewidth=1.5)
axes[3].axhline(0, color='black', linestyle='-', alpha=0.5)
axes[3].set_ylabel('B_N [nT]')
axes[3].set_xlabel('Time [hours from start]')
axes[3].grid(True, alpha=0.3)

# Mark crossings on all panels
if xings:
    for t_cross, cross_type in xings:
        t_cross_hours = (t_cross - t_grid[0]) / 3600
        color = 'red' if 'enter' in cross_type.lower() else 'blue'
        for ax in axes:
            ax.axvline(t_cross_hours, color=color, linestyle=':', alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

print("\n📊 Plot shows:")
print("  • Magnetic field in LMN coordinates")
print("  • Ion and electron densities")
print("  • He⁺ density with detection thresholds")
print("  • B_N component with crossing markers")
print("  • Red lines: magnetosphere entry")
print("  • Blue lines: magnetosheath entry")

## Summary

This notebook demonstrated the basic workflow of the MMS Magnetopause Analysis Toolkit:

1. **Data Loading**: Automatic download and loading of MMS Level-2 data
2. **Resampling**: Uniform time grid for multi-instrument analysis
3. **Coordinates**: Hybrid LMN transformation combining MVA and model
4. **Detection**: Multi-parameter boundary detection using He⁺ and B_N
5. **Visualization**: Summary plots showing key parameters

## Next Steps

- Try `02_advanced_analysis.ipynb` for multi-spacecraft timing analysis
- Experiment with different time ranges and detection parameters
- Use the CLI for batch processing: `python -m mms_mp.cli --help`